In [14]:
import nltk
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gensim.downloader as api
from gensim.test.utils import datapath
import gensim
import math
import re
import random
import numpy as np
import sys
from unidecode import unidecode

questions_train = []
tables_train = []
actual_col_train = []
label_cols_train = []
label_rows_train = []
with open('data/A2_train.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        parsed_data = json.loads(line)
        questions_train.append(parsed_data['question'])
        tables_train.append(parsed_data['table'])
        label_cols_train.append(parsed_data['label_col'][0])
        actual_col_train.append(list(parsed_data['table']['cols']))
        label_rows_train.append(list(parsed_data['label_row']))

# questions_train = clean_questions(questions_train)

print('Number of questions:', len(questions_train))
print('Number of tables:', len(tables_train))
print('Number of label columns:', len(label_cols_train))
print('Number of actual columns:', len(actual_col_train))
print('Number of label rows:', len(label_rows_train))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

questions_test = []
tables_test = []
actual_col_test = []
label_cols_test = []
label_rows_test = []
qid_test = []

with open('data/A2_val.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        parsed_data = json.loads(line)
        questions_test.append(parsed_data['question'])
        tables_test.append(parsed_data['table'])
        label_cols_test.append(parsed_data['label_col'][0])
        actual_col_test.append(list(parsed_data['table']['cols']))
        qid_test.append(parsed_data['qid'])
        label_rows_test.append(list(parsed_data['label_row']))

# questions_test = clean_questions(questions_test)

print('Number of questions:', len(questions_test))
print('Number of tables:', len(tables_test))
print('Number of label columns:', len(label_cols_test))
print('Number of actual columns:', len(actual_col_test))
print('Number of qids is ', len(qid_test))
print('Number of label rows:', len(label_rows_test))

Number of questions: 25000
Number of tables: 25000
Number of label columns: 25000
Number of actual columns: 25000
Number of label rows: 25000
Number of questions: 5000
Number of tables: 5000
Number of label columns: 5000
Number of actual columns: 5000
Number of qids is  5000
Number of label rows: 5000


In [42]:
train_set = []
label_set = []
total_num_row =0
for i in range(len(questions_train)):
    question = unidecode(questions_train[i]).lower()
    table = tables_train[i]
    actual_col = actual_col_train[i]
    label_row = label_rows_train[i]
    for j in range(len(table['rows'])):
        total_num_row+=1
        row = table['rows'][j]
        score = torch.empty(3)
        for k in range(len(row)):
            cell = unidecode(row[k]).lower()
            that_col = unidecode(actual_col[k]).lower()
            score[0]= score[0]+len(cell)
            score[1]= score[1]+len(cell)*len(cell)
            score[2]= score[2]+len(cell)*len(cell)*len(cell)
        train_set.append(score)
        if j in label_row:
            label_set.append([0,1])
        else:
            label_set.append([1,0])



In [43]:
label_set = torch.tensor(label_set, dtype=torch.float32)
train_set = torch.stack(train_set)


In [44]:
class neuralNet(nn.Module):
    def __init__(self):
        super(neuralNet,self).__init__()
        self.first_layer = nn.Linear(3, 10)
        self.relu = nn.ReLU()
        self.second_layer = nn.Linear(10, 25)
        self.tanh = nn.Tanh()
        self.third_layer = nn.Linear(25, 2)

    def forward(self, x):
        x = self.first_layer(x)
        x = self.relu(x)
        x = self.second_layer(x)
        x = self.tanh(x)
        x = self.third_layer(x)
        return x

In [ ]:
row_model = neuralNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(row_model.parameters(), lr=0.005)
row_model.train()
epochs = 100
for i in range(epochs):
    # model.train()
    acc_train = 0
    optimizer.zero_grad()
    outputs = row_model(train_set)
    # print(outputs.shape)
    # print(label_set.shape)
    loss = criterion(outputs,label_set)
    acc_train += (outputs.argmax(dim=1) == label_set.argmax(dim=1)).sum().item()
    loss.backward()
    optimizer.step()
    # print('acc is ', acc_train/total_num_row)
    row_model.eval()
